# 神经网络量化白皮书(QAT)

## 4 量化感知训练

上一节中描述的后训练量化技术是我们量化工具包中的首选工具。它们非常有效且易于实现，因为它们不需要使用带标签的数据重新训练网络。然而，它们存在一些限制，特别是当目标是对激活进行低比特量化（例如 4 位及以下）时。后训练技术可能不足以缓解低比特量化引入的巨大量化误差。在这种情况下，我们采用量化感知训练（QAT）。QAT 在训练期间模拟量化噪声源（参见第 2.3 节）。这使得模型能够找到比后训练量化更优化的解决方案。然而，更高的精度伴随着神经网络训练的常见成本，即更长的训练时间、需要带标签的数据和超参数搜索。

在本节中，我们探讨了如何在具有模拟量化的网络中进行反向传播，并为有效地训练使用 QAT 的模型提供了标准流程。我们还将讨论批量归一化折叠和通道级量化在 QAT 中的影响，并提供各种任务和模型的结果。

### 4.1 模拟量化用于反向路径

在 2.3 节中，我们看到了如何使用深度学习框架中的浮点数来模拟量化。然而，如果我们看一下 {numref}`fig4` 的计算图，要训练这样的网络，我们需要通过模拟量化器块进行反向传播。这引发了一个问题，因为方程 {eq}`q4` 中的四舍五入操作的梯度在任何地方都是零或未定义的，这使得基于梯度的训练变得不可能。解决这个问题的一种方法是使用直通估计器（straight-through estimator，简称 STE，Bengio等人，2013年），它将舍入运算符的梯度近似为 $1$：

$$
\cfrac{\partial{\lfloor y \rceil}}{\partial{y}} = 1
$$